### Predicting Mask on Augmented Dataset

1. Load Model
2. Load Test Dataset - Images
3. Predict Mask
4. Calculate FPS

In [2]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

#### Seeding

In [3]:
# Seeding for reproducibility
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

#### Hyperparameters

In [4]:
# Hyperparameters
height = 512
width = 768
num_classes = 3

#### Path

In [5]:
# Paths
dataset_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/processed_dataset/aug/test"
save_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/predicted_masks/ResUnet"
model_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/files/ResUnet/resunet-multiclass.keras"

#### Create Folder

In [6]:
os.makedirs(save_path, exist_ok=True)

#### Load Model

In [7]:
# Load trained model
model = tf.keras.models.load_model(model_path, compile=False)

In [8]:
model.summary()

Model: "ResU-Net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 768,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 512, 768,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512, 768,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 512, 768,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 512, 768,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 512, 768,  │        256 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 768,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 768,  │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 512, 768,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 512, 768,  │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 256, 384,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 256, 384,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 384,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 256, 384,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 256, 384,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 256, 384,  │      8,320 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 384,  │        512 │ conv2d_4[0][0]  

 Total params: 32,462,979 (123.84 MB)

 Trainable params: 32,445,315 (123.77 MB)

 Non-trainable params: 17,664 (69.00 KB)

#### Load Test Dataset: Images

In [9]:
# Load test images
test_x = sorted(glob(os.path.join(dataset_path, "images", "*.png")))

In [10]:
# Mapping grayscale values to class labels
class_colors = {
    0: 0,   # Non-Drivable Area (black)
    1: 128, # My Way (gray level 128)
    2: 255  # Other Way (white)
}

# Track inference times
time_taken = []

for x_path in tqdm(test_x, desc="Predicting Masks"):
    name = os.path.basename(x_path)
    x = cv2.imread(x_path, cv2.IMREAD_COLOR)

    if x is None:
        print(f"Error loading image {x_path}, skipping.")
        continue

    x = cv2.resize(x, (width, height))
    x = x / 255.0
    x = np.expand_dims(x, axis=0)

    # Predict mask using ResU-Net model
    start_time = time.time()
    p = model.predict(x)[0]
    time_taken.append(time.time() - start_time)

    p = np.argmax(p, axis=-1).astype(np.uint8)

    print(f"Unique predicted values for {name}: {np.unique(p)}")  # Debugging

    # Create a grayscale mask
    p_gray = np.zeros((height, width), dtype=np.uint8)

    # Assign grayscale values based on class labels
    for label, gray_value in class_colors.items():
        p_gray[p == label] = gray_value

    # Save predicted mask in grayscale
    cv2.imwrite(os.path.join(save_path, f"{name.split('.')[0]}.png"), p_gray, [cv2.IMWRITE_PNG_COMPRESSION, 0])

# FPS Calculation
mean_time = np.mean(time_taken)
mean_fps = 1 / mean_time
print(f"Mean time taken per image: {mean_time:.4f} seconds")
print(f"Mean FPS: {mean_fps:.2f}")

print(f"Predicted masks saved in {save_path}")

Predicting Masks:   0%|          | 0/19 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


Predicting Masks:   5%|▌         | 1/19 [00:01<00:22,  1.26s/it]

Unique predicted values for 0008.png: [0 1 2]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 776ms/step


Predicting Masks:  11%|█         | 2/19 [00:02<00:16,  1.01it/s]

...

Unique predicted values for 0104.png: [0 1 2]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 784ms/step


Predicting Masks:  95%|█████████▍| 18/19 [00:15<00:00,  1.23it/s]

Unique predicted values for 0109.png: [0 1 2]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 790ms/step


Predicting Masks: 100%|██████████| 19/19 [00:15<00:00,  1.20it/s]

Unique predicted values for 0115.png: [0 1 2]
Mean time taken per image: 0.8188 seconds
Mean FPS: 1.22
Predicted masks saved in /home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/predicted_masks/ResUnet
